# 飞桨新人赛：钢铁缺陷检测挑战赛-第10名方案
[比赛链接](https://aistudio.baidu.com/aistudio/competition/detail/114) https://aistudio.baidu.com/aistudio/competition/detail/114

###  过程：
1.使用了faster-rcnn, 训练了300个epoches, 检测结果用了threshold<0.5 去除检测框， 分数36分左右（这里提交csv有个小插曲, 就是检测结果必须按照image_id的顺序排列， 后台判分就是按照image_id来判分的）

2. 参照别人的经验，使用ppyolo v2的模型， 训练500 epoches, 使用threshold<0.05 去除检测框,分数42.06
尝试平均两个模型结果， 分数<36
直接拼接两个模型结果，分数<36

###  感受：
后面工作比较忙就没有继续提升了。还是要赞一下paddle平台， 恢复训练会继续上一次epoches往下计数(也就是lr 也是延续原来的策略)， 对比我之前用一些github上的源码工程， 恢复训练总是会重新计数，就很蠢。 



In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleDetection.git

Cloning into 'PaddleDetection'...
remote: Enumerating objects: 18212, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 18212 (delta 15), reused 38 (delta 12), pack-reused 18162
Receiving objects: 100% (18212/18212), 181.60 MiB | 11.27 MiB/s, done.
Resolving deltas: 100% (13158/13158), done.
Checking connectivity... done.


### 对数据集进行分析统计

In [ ]:
!pip install lxml


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 5.5MB 2.7MB/s eta 0:00:01


In [ ]:
!python  ~/work/analyse_voc.py
#0.3， 0.5， 1， 2， 4， 12

anno_set ['rolled-in_scale', 'patches', 'crazing', 'pitted_surface', 'inclusion', 'scratches']
num_list [493, 679, 537, 339, 788, 420]


In [ ]:
!python  ~/work/object-detection-anchors/example.py

Start to load data annotations on: ./data/widerface-annotations
Start to do kmeans, please wait for a moment.
Traceback (most recent call last):
  File "/home/aistudio/work/object-detection-anchors/example.py", line 104, in <module>
    out = kmeans(data, k=CLUSTERS)
  File "/home/aistudio/work/object-detection-anchors/kmeans.py", line 72, in kmeans
    clusters = boxes[np.random.choice(rows, k, replace=False)]
  File "mtrand.pyx", line 903, in numpy.random.mtrand.RandomState.choice
ValueError: a must be greater than 0 unless no samples are taken


In [ ]:
# 使用x2coco.py文件将voc格式转化为coco(失败)
!python ~/work/PaddleX-develop/paddlex/tools/dataset_conversion/x2coco.py \
#!python work/x2coco.py \
--dataset_type voc \
--image_input_dir work/train/IMAGES \
--voc_anno_dir Pwork/train/ANNOTATIONS \
#--voc_anno_list PaddleDetection/dataset/VOC2012/ImageSets/Main0/test.txt \
#--voc_label_list PaddleDetection/dataset/VOC2012/ImageSets/Main/label_list.txt \
--output_dir work/coco/train \
--voc_out_name 'test.json'


### 准备配置文件

PaddleX的使用需要提供分类标签文件`label.txt`

![](https://ai-studio-static-online.cdn.bcebos.com/ec19891595a34046937b165922cfaed0928ee30ea4704555b25a2e94ea01d64e)

以及图片-xml的链接文件`train_list.txt` `val_list.txt`，分别用于验证和训练

![](https://ai-studio-static-online.cdn.bcebos.com/1de87d9972e44eb0b1136a21590c0ffd4a0c5f38b47a4433aa07583b589dca5b)

下面将遍历训练图片并生成对应的`train_list.txt` `val_list.txt`

In [ ]:
# 解压文件并移除多余的目录
! unzip work/train.zip -d ./work/
! rm -f -r work/__MACOSX
! unzip work/test.zip -d ./work/
! rm -f -r work/__MACOSX

In [ ]:
import os
%cd /home/aistudio/
# 遍历训练集
name = [name for name in os.listdir('work/train/IMAGES') if name.endswith('.jpg')]

train_name_list=[]
for i in name:
    # Tag:
    tmp = os.path.splitext(i)
    train_name_list.append(tmp[0])

# 构造图片-xml的链接文件ori_train.txt
with open("./work/train/ori_train.txt","w") as f:
    for i in range(len(train_name_list)):
        if i!=0: f.write('\n')
        line='IMAGES/'+train_name_list[i]+'.jpg'+" "+"ANNOTATIONS/"+train_name_list[i]+'.xml' 
        f.write(line)

# 构造label.txt
labels=['crazing','inclusion','pitted_surface','scratches','patches','rolled-in_scale']
with open("./work/train/labels.txt","w") as f:
    for i in range(len(labels)):
        line=labels[i]+'\n'
        f.write(line)

# 将ori_train.txt随机按照eval_percent分为验证集文件和训练集文件
# eval_percent 验证集所占的百分比
import random
eval_percent=0.2;

data=[]
with open("work/train/ori_train.txt", "r") as f:
    for line in f.readlines():
        line = line.strip('\n')
        data.append(line)

index=list(range(len(data)))
random.shuffle(index)

# 构造验证集文件
cut_point=int(eval_percent*len(data))
with open("./work/train/val_list.txt","w") as f:
    for i in range(cut_point):
        if i!=0: f.write('\n')
        line=data[index[i]]
        f.write(line)

# 构造训练集文件
with open("./work/train/train_list.txt","w") as f:
    for i in range(cut_point,len(data)):
        if i!=cut_point: f.write('\n')
        line=data[index[i]]
        f.write(line)

/home/aistudio


In [ ]:
import os
%cd /home/aistudio/
# 遍历训练集
name = [name for name in os.listdir('work/test/IMAGES') if name.endswith('.jpg')]

train_name_list=[]
for i in name:
    tmp = os.path.splitext(i)
    train_name_list.append(tmp[0])

# 构造图片-xml的链接文件ori_train.txt
with open("./work/test/val_list.txt","w") as f:
    for i in range(len(train_name_list)):
        if i!=0: f.write('\n')
        line='IMAGES/'+train_name_list[i]+'.jpg'+" "+"ANNOTATIONS/"+train_name_list[i]+'.xml' 
        f.write(line)

/home/aistudio


## 正式进行训练工作
### 使用--eval 会默认每训练完一个epoch就对验证集进行一次预测。

In [1]:
#安装依赖
%cd PaddleDetection/
!pip install scikit-image
!pip install -r requirements.txt


/home/aistudio/PaddleDetection
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 13.3MB 8.1MB/s eta 0:00:01
     |████████████████████████████████| 184kB 4.9MB/s eta 0:00:01
     |████████████████████████████████| 6.2MB 5.4MB/s eta 0:00:01
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.1MB 5.3MB/s eta 0:00:01
     |████████████████████████████████| 112kB 426kB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 3.1MB/s eta 0:00:01
     |████████████████████████████████| 153kB 3.4MB/s eta 0:00:01
     |████████████████████████████████| 51kB 2.8MB/s eta 0:00:011
     |████████████████████████████████| 286kB 998kB/s eta 0:00:01
     |████████████████████████████████| 51kB 210kB/s eta 0:00:011
     |████████████████████████████████| 102kB 1.5MB/s ta 0:00:011
     |████████████████████████████████| 102kB 1.3MB/s ta 0:00:011
  Created wheel for pycocotools: filename=pycocotools-

In [2]:
%cd PaddleDetection/
%env CUDA_VISIBLE_DEVICES=0
# -c /home/aistudio/PaddleDetection/configs/faster_rcnn/faster_rcnn_dcn_x101_vd_64x4d_fpn_1x.yml \
#使用visualDL记录曲线变化
!python -u tools/train.py \
    -c /home/aistudio/PaddleDetection/configs/ppyolo/ppyolov2_r50vd_dcn_voc.yml\
    --use_vdl=True \
    --vdl_log_dir=vdl_dir/scalar \
    -r output/ppyolov2_r50vd_dcn_voc/331.pdparams \
    --eval 
    
'''    
!python -u tools/train.py \
    -c /home/aistudio/PaddleDetection/configs/faster_rcnn/faster_rcnn_dcn_x101_vd_64x4d_fpn_1x.yml \
    --use_vdl=True \
    --vdl_log_dir=vdl_dir/scalar \
    --eval
'''
    #-r output/faster_rcnn_r50_fpn_2x/22000.pdparams

'    \n!python -u tools/train.py     -c /home/aistudio/PaddleDetection/configs/faster_rcnn/faster_rcnn_dcn_x101_vd_64x4d_fpn_1x.yml     --use_vdl=True     --vdl_log_dir=vdl_dir/scalar     --eval\n'

### 手动对验证集进行预测



In [4]:
%cd PaddleDetection/
# !python tools/eval.py \
#     -c /home/aistudio/PaddleDetection/configs/faster_rcnn/faster_rcnn_dcn_x101_vd_64x4d_fpn_1x.yml \
#     -o weights=output/faster_rcnn_dcn_x101_vd_64x4d_fpn_1x/model_final.pdparams \
#     save_prediction_only=True

# o weights=output/ppyolov2_r50vd_dcn_voc/best_model.pdparams \
!python tools/eval.py \
    -c /home/aistudio/PaddleDetection/configs/ppyolo/ppyolov2_r50vd_dcn_voc.yml \
    -o weights=output/ppyolov2_r50vd_dcn_voc/82.pdparams \
    save_prediction_only=True

[Errno 2] No such file or directory: 'PaddleDetection/'
/home/aistudio/PaddleDetection
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/openpyxl/compat/numbers.py:41: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  numpy.

### 对test集进行推理预测， 
1. 对测试集生成一份假的json标注文件（test测试集本身没有标注）
2. 这里修改了Paddle Detection 的源码tools/infer.py与engine/ppdet/trainer.py, 修改是为了把预测结果输出为一个json文件
(参考链接：https://blog.csdn.net/qq_40502460/article/details/117826384)

In [ ]:
!python ~/work/generate_fake_test_json.py

1
2
3
4
5
6


In [ ]:
##############对eval 测试集！！！
%cd PaddleDetection/
!python tools/infer.py -c /home/aistudio/PaddleDetection/configs/faster_rcnn/faster_rcnn_dcn_x101_vd_64x4d_fpn_1x.yml \
                    --infer_dir=/home/aistudio/work/train/eval \
                    --output_dir=infer_output_eval/ \
                    --draw_threshold=0.5 \
                    -o weights=output/faster_rcnn_dcn_x101_vd_64x4d_fpn_1x/model_final.pdparams  \
                    --save_txt=True
                    #--use_vdl=Ture

In [6]:
%cd PaddleDetection/
# ##########魔改后， svae_txt=True代表不生成json文件！， 反之 生成json文件。 
# ##########此方法不需要instance_test.json. 
!python tools/infer.py -c /home/aistudio/PaddleDetection/configs/faster_rcnn/faster_rcnn_dcn_x101_vd_64x4d_fpn_1x.yml \
                    --infer_dir=/home/aistudio/work/test/IMAGES \
                    --output_dir=/home/aistudio/data/infer_output/ \
                    --draw_threshold=0.005 \
                    -o weights=output/faster_rcnn_dcn_x101_vd_64x4d_fpn_1x/model_final.pdparams  
                    # --save_txt=True

# !python tools/infer.py     -c /home/aistudio/PaddleDetection/configs/ppyolo/ppyolov2_r50vd_dcn_voc.yml \
#     -o weights=output/ppyolov2_r50vd_dcn_voc/best_model.pdparams \
#                     --infer_dir=/home/aistudio/work/test/IMAGES \
#                     --output_dir=/home/aistudio/data/infer_output/ \
#                     --draw_threshold=0.005 \



[Errno 2] No such file or directory: 'PaddleDetection/'
/home/aistudio/PaddleDetection
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/openpyxl/compat/numbers.py:41: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  numpy.

### json转csv， 用于结果提交

In [7]:
import csv
import json

'''
json格式示例
[{ "firstName":"Bill" , "lastName":"Gates" }, { "firstName":"George" , "lastName":"Bush" }, { "firstName":"Thomas" , "lastName":"Carter" }]
'''
#file:json to csv
def transcsv(jsonpath, csvpath, threshold):
    json_file = open(jsonpath, 'r', encoding='utf8')
    csv_file = open(csvpath, 'w', newline='')
    #writer = csv.writer(csv_file, delimiter='\t')
    writer = csv.writer(csv_file, delimiter=',')
    #读文件
    ls = json.load(json_file)  #将json格式的字符串转换成python的数据类型，解码过程
    # data = [list(ls[0].keys())]  # 获取列名,即key
    # 只需要把是int的value转为str
    order = ["image_id","bbox","category_id","confidence"]
    first_line=["image_id","bbox","category_id","confidence"]
    data=[first_line]
    for item in ls:
        #print("item[\"score\"]", item["score"])
        if item["confidence"]<threshold:
            continue
        else:
            value_list=[]
            for itm in order:
                if(itm =="bbox"):
                    # 处理
                    tmp=item[itm]
                    tmp[2]+=tmp[0]
                    tmp[3]+=tmp[1]
                    #print("type(tmp[2])", type(tmp[2]))
                    value_list.append(tmp)
                else:
                    value_list.append(item[itm])
                # print(value_list)
            data.append(value_list)

            # for itm in item.values():
            #     # print("type(itm)", type(itm))
            #     if isinstance(itm, list):
            #         # print("is list")
                    
            #         tmp=itm
            #         #print("before tmp", tmp)
            #         tmp[2]+=tmp[0]
            #         tmp[3]+=tmp[1]
            #         #print("type(tmp[2])", type(tmp[2]))
            #         value_list.append(tmp)
            #         #print("after tmp", tmp)
            #     else:
            #         value_list.append(itm)
            # data.append(value_list)
        # data.append(list(item.values()))  # 获取每一行的值value
        
    #写入文件
    for line in data:
        # print("line", line)
        # csv_file.write(",".join(line) + "\n")  # 以逗号分隔一行的每个元素，最后换行 fw.close() #关闭csv文件
        writer.writerow(line)


    #关闭文件
    json_file.close()
    csv_file.close()
#file:csv to json
def transjson(jsonpath, csvpath):
    fw = open(jsonpath, 'w', encoding='utf8')   # 打开csv文件
    fo = open(csvpath, 'r', newline='')    # 打开csv文件

    ls = []
    for line in fo:
        line = line.replace("\n", "")  # 将换行换成空
        ls.append(line.split(","))  # 以，为分隔符
    #print(ls)
    #写入
    for i in range(1, len(ls)):  # 遍历文件的每一行内容，除了列名
        ls[i] = dict(zip(ls[0], ls[i]))  # ls[0]为列名，所以为key,ls[i]为value,
        # zip()是一个内置函数，将两个长度相同的列表组合成一个关系对

    json.dump(ls[1:], fw, sort_keys=True, indent=4)
    #将Python数据类型转换成json格式，编码过程
    #默认是顺序存放，sort_keys是对字典元素按照key进行排序
    #indet参数用语增加数据缩进，使文件更具有可读性

    # 关闭文件
    fo.close()
    fw.close()
# 实际上只需要处理csv文件。 threshold》0.5 的保留。 

# if __name__ == '__main__':
#     #transcsv('./testcase/my.json', './testcase/my.csv')
#     transjson('./testcase/write.json', './testcase/my.csv')

In [8]:
# trans('/home/aistudio/work/test/result/result.json', '/home/aistudio/work/test/result/submission.csv')

# json.loads读的是单行数据， json.load读的是文件， 多行数据
# data= json.loads('/home/aistudio/work/test/result/result.json')
# print(data)

# with open('/home/aistudio/work/test/result/result.json') as f:
#     result=json.load(f)
#     print("type(result)", type(result))

transcsv('/home/aistudio/work/test/result/result.json', '/home/aistudio/work/test/result/submission.csv',0)
# transcsv('/home/aistudio/PaddleDetection-release-2.0-rc/bbox.json', '/home/aistudio/work/test/result/submission.csv',0)
# transjson('/home/aistudio/paddledetection2-3/submission.json','/home/aistudio/paddledetection2-3/submission.csv')

In [11]:
######对csv文件进行threshold后处理（筛选
import pandas as pd
'''
csv_path='/home/aistudio/paddledetection2-3/submission.csv'
csv_process_path='/home/aistudio/paddledetection2-3/submission_process.csv'

data = pd.read_csv(csv_path)
# print("data", data["confidence"])
data2=data[data["confidence"]>=0.5] 
data2.to_csv(csv_process_path, index=False)
# print("data > 0.5", data["confidence"]>=0.5)
'''

#####修改confidence =1
# csv_path="/home/aistudio/work/test/result/submission.csv"
# data = pd.read_csv(csv_path)
# # print("data", data["confidence"])
# # data2=data[data["confidence"]>=0.5] 
# data["confidence"]=1.0
# data.to_csv(csv_path, index=False)

#########整理csv的顺序

# csv_path="/home/aistudio/work/test/result/submission.csv"
# # csv_path="/home/aistudio/work/fusion_submission.csv"
# data = pd.read_csv(csv_path)
# # print("data", data)
# sortedlist= data.sort_values(by="image_id")
# sortedlist.to_csv(csv_path, index=False)


import json
# Tag: 两个csv先相加融合, 然后排序， 生成 concat_submission.csv。 
# image_id.values, distinct找到
csv_path="/home/aistudio/work/test/result/submission.csv"
csv_path1="/home/aistudio/work/test/submission42.06.csv"
concat_path="/home/aistudio/work/test/result/concat_submission.csv"
data = pd.read_csv(csv_path)
data1 = pd.read_csv(csv_path1)
data = pd.concat([data, data1], ignore_index = True)
sortedlist= data.sort_values(by="image_id")
sortedlist.to_csv(concat_path, index=False)
# print("data", data)

'''
import json
# Tag: 直接两个csv先相加融合。 
# image_id.values, distinct找到
csv_path="/home/aistudio/work/test/result/submission.csv"
csv_path1="/home/aistudio/paddledetection2-3/submission_process.csv"
data = pd.read_csv(csv_path)
data1 = pd.read_csv(csv_path1)
# 可以"a"的形式写成文件。 
# for 
#     df.to_csv(filename, mode='a'）
data = pd.concat([data, data1], ignore_index = True)
# print("data", data)
df_img_id=data["image_id"].unique()
# print("df_img_id", df_img_id)
# 遍历image_id,
all_ids = []
all_boxes = []
all_scores = []
all_labels = []
for img_id in df_img_id:
    # 组建boxes, scores, labels _lists
    # print("img_id", img_id)
    boxes_list = []
    scores_list = []
    labels_list = []
    for idx,ix in data[data["image_id"]==int(img_id)].iterrows():
        # print("ix", ix)
        # bbox = json.loads(ix)
        
        box = json.loads(ix["bbox"])
        score = ix["confidence"]
        label = ix["category_id"]
        boxes_list.append(box)
        scores_list.append(score)
        labels_list.append(label)
    #然后weighted_boxes_fusion， 然后结果放入dataFrame
    if run_type == 'wbf':
        merged_boxes, merged_scores, merged_labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list,
                                                                           weights=weights,
                                                                           iou_thr=params['intersection_thr'],
                                                                           skip_box_thr=params['skip_box_thr'],
                                                                           conf_type=params['conf_type']) 

    id_list=[img_id] *len(merged_labels)  
    all_boxes.append(merged_boxes)
    all_scores.append(merged_scores)
    all_labels.append(merged_labels)
    all_ids.append(ids_list)
#根据dataFrame生成一份csv
res = pd.DataFrame(all_ids, columns=['image_id'])
res['bbox'] = all_boxes
res['category_id'] = all_labels
res['confidence'] = all_scores
'''


'\nimport json\n# Tag: 直接两个csv先相加融合。 \n# image_id.values, distinct找到\ncsv_path="/home/aistudio/work/test/result/submission.csv"\ncsv_path1="/home/aistudio/paddledetection2-3/submission_process.csv"\ndata = pd.read_csv(csv_path)\ndata1 = pd.read_csv(csv_path1)\n# 可以"a"的形式写成文件。 \n# for \n#     df.to_csv(filename, mode=\'a\'）\ndata = pd.concat([data, data1], ignore_index = True)\n# print("data", data)\ndf_img_id=data["image_id"].unique()\n# print("df_img_id", df_img_id)\n# 遍历image_id,\nall_ids = []\nall_boxes = []\nall_scores = []\nall_labels = []\nfor img_id in df_img_id:\n    # 组建boxes, scores, labels _lists\n    # print("img_id", img_id)\n    boxes_list = []\n    scores_list = []\n    labels_list = []\n    for idx,ix in data[data["image_id"]==int(img_id)].iterrows():\n        # print("ix", ix)\n        # bbox = json.loads(ix)\n        \n        box = json.loads(ix["bbox"])\n        score = ix["confidence"]\n        label = ix["category_id"]\n        boxes_list.append(box)\n        scor

## ===== 以下是我更换ppdet版本之后进行的一些尝试， 无需理会！==============

In [ ]:
########################### 降级为ppdet2.0进行训练测试。 
#安装依赖
%cd PaddleDetection-release-2.0-rc/
!pip install pycocotools
!pip install scikit-image
!pip install -r requirements.txt

In [ ]:
%cd PaddleDetection-release-2.0-rc/
%env CUDA_VISIBLE_DEVICES=0
#使用visualDL记录曲线变化
!python -u tools/train.py \
    -c configs/libra_rcnn/libra_rcnn_r50_vd_fpn_1x.yml \
    --use_vdl=True \
    --vdl_log_dir=vdl_dir/scalar \
    -r output/libra_rcnn_r50_vd_fpn_1x/10000.pdparams \
    # -o weights=output/libra_rcnn_r50_vd_fpn_1x/10000.pdparams \
    --eval

In [ ]:
#############可以直接用此方法保存预测的json文件， 但是需要修改yaml . 
%cd PaddleDetection-release-2.0-rc/
!python tools/eval.py \
    -c configs/libra_rcnn/libra_rcnn_r50_vd_fpn_1x_infer.yaml \
    -o weights=output/libra_rcnn_r50_vd_fpn_1x/model_final.pdparams \
    save_prediction_only=True


In [ ]:
# 预览预测的结果, 需要改一改label . 
# 代码不同， 修改infer, trainer代价高。 
%cd PaddleDetection-release-2.0-rc/
!python tools/infer.py     -c configs/libra_rcnn/libra_rcnn_r50_vd_fpn_1x_infer.yaml \
                    -o weights=output/libra_rcnn_r50_vd_fpn_1x/model_final.pdparams \
                    --infer_dir=/home/aistudio/work/test/IMAGES \
                    --output_dir=infer_output/ \
                    --draw_threshold=0.6 \
                    # -o weights=output/faster_rcnn_dcn_x101_vd_64x4d_fpn_1x/model_final.pdparams  \
                    #--save_txt=True
                    #--use_vdl=Ture

降级为ppdet2.3进行测试。 


In [ ]:
%cd paddledetection2-3/PaddleDetection/
!pip install pycocotools
!pip install scikit-image
!pip install -r requirements.txt

/home/aistudio/paddledetection2-3/PaddleDetection


In [ ]:
!python tools/infer.py -c configs/ppyolo/ppyolov2_r50vd_dcn_voc.yml \
-o weights=../ppyolov2_r50vd_dcn_voc/1394 \
--infer_img=/home/aistudio/work/test/IMAGES/1400.jpg \
--output_dir=/home/aistudio/data/steel/infer_output\
--draw_threshold=0.005 --save_txt=True

# --draw_threshold=0.5 --save_txt=True
# --infer_img=/home/aistudio/work/test/IMAGES/1400.jpg \
# --infer_dir=/home/aistudio/work/test/IMAGES \